In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import json

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(30)

model_id = "deepseek-ai/DeepSeek-Prover-V2-7B"  # or DeepSeek-Prover-V2-671B
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0", torch_dtype=torch.bfloat16, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

`rope_scaling`'s factor field must be a float >= 1, got 16
`rope_scaling`'s beta_fast field must be a float, got 32
`rope_scaling`'s beta_slow field must be a float, got 1


model.safetensors.index.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.60G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### CoT Generation

In [ ]:
formal_statement = """
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
""".strip()


prompt = """
Complete the following Lean 4 code:

```lean4
{}
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.
""".strip()

chat = [
  {"role": "user", "content": prompt.format(formal_statement)},
]
inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)



In [3]:
import time
start = time.time()
outputs = model.generate(inputs, max_new_tokens=8192)
print(tokenizer.batch_decode(outputs))
print(time.time() - start)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:\n\n```lean4\nimport Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n/-- What is the positive difference between $120\\%$ of 30 and $130\\%$ of 20? Show that it is 10.-/\ntheorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by\n  sorry\n```\n\nBefore producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.\nThe plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.<｜Assistant｜>### Detailed Proof and Analysis\n\nFirst, we need to understand the problem:\n\n**Problem:** What is the positive difference between \\(120\\%\\) of \\(30\\) and \\(130\\%\\) of \\(20\\)? Show that it is \\(10\\).\n\n**Solution:**\n\n1. **Calculate \\(120\\%\\) of \\(30\\):**\n   \\[\n   \\frac{120}{100} \\times 3

In [7]:
text = tokenizer.batch_decode(outputs)[0]
print(text)

<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.<｜Assistant｜>### Detailed Proof and Analysis

First, we need to understand the problem:

**Problem:** What is the positive difference between \(120\%\) of \(30\) and \(130\%\) of \(20\)? Show that it is \(10\).

**Solution:**

1. **Calculate \(120\%\) of \(30\):**
   \[
   \frac{120}{100} \times 30 = \frac{120 \times 30}{100} = \frac{3600}{100} =

### Non-CoT Generation

In [9]:
formal_statement = """
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
""".strip()


prompt = """
Complete the following Lean 4 code:

```lean4
{}
```
""".strip()


chat = [
  {"role": "user", "content": prompt.format(formal_statement)},
]
inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)

In [10]:
import time
start = time.time()
outputs = model.generate(inputs, max_new_tokens=8192)
print(tokenizer.batch_decode(outputs)[0])
print(time.time() - start)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- What is the positive difference between $120\%$ of 30 and $130\%$ of 20? Show that it is 10.-/
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  sorry
```<｜Assistant｜>```lean4
theorem mathd_algebra_10 : abs ((120 : ℝ) / 100 * 30 - 130 / 100 * 20) = 10 := by
  norm_num [abs_eq_self.mpr, abs_eq_neg_self.mpr, div_eq_mul_inv, mul_assoc]
  <;> ring_nf
  <;> norm_num
  <;> apply abs_of_nonneg
  <;> norm_num
  <;> apply abs_of_nonneg
  <;> norm_num
  <;> apply abs_of_nonneg
  <;> norm_num
```<｜end▁of▁sentence｜>
2.7036092281341553


### Python Interface for Lean4

In [ ]:
from lean_interact import LeanREPLConfig, LeanServer, Command, FileCommand

config = LeanREPLConfig(verbose=True) # download and build Lean REPL
server = LeanServer(config) # start Lean REPL

# Execute a Lean file
response = server.run(FileCommand(path="myfile.lean"))

### Execute Lean to Verify Correctness

In [1]:
lean_code = """
theorem my_theorem : 1 + 1 = 2 :=
by rfl
"""

with open("verify_this.lean", "w") as f:
    f.write(lean_code)

In [3]:
import subprocess

lean_dir = "/data2/haikang/projects/math-rl/lean-project"
# run subprocess in lean_dir
process = subprocess.run(["lake", "env", "lean", "verify_this.lean"], cwd=lean_dir, capture_output=True, text=True)

if process.returncode == 0:
    print("Lean code verification successful!")
else:
    print("Lean code verification failed:")
    print(process.stderr)

Lean code verification successful!


## Magma

In [10]:
formal_statement = """
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

class Magma (α : Type _) where
  op : α → α → α

infix:65 " ◇ " => Magma.op

abbrev Equation2 (G: Type _) [Magma G] := ∀ x y : G, x = y

abbrev Equation4 (G: Type _) [Magma G] := ∀ x y : G, x = x ◇ y

theorem Equation2_implies_Equation4 (G: Type _) [Magma G] (h: Equation2 G) : Equation4 G := by
  sorry
""".strip()


prompt = """
Complete the following Lean 4 code:

```lean4
{}
```
""".strip()


chat = [
  {"role": "user", "content": prompt.format(formal_statement)},
]
inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)


In [11]:
import time
start = time.time()
outputs = model.generate(inputs, max_new_tokens=8192)
print(tokenizer.batch_decode(outputs)[0])
print(time.time() - start)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

class Magma (α : Type _) where
  op : α → α → α

infix:65 " ◇ " => Magma.op

abbrev Equation2 (G: Type _) [Magma G] := ∀ x y : G, x = y

abbrev Equation4 (G: Type _) [Magma G] := ∀ x y : G, x = x ◇ y

theorem Equation2_implies_Equation4 (G: Type _) [Magma G] (h: Equation2 G) : Equation4 G := by
  sorry
```<｜Assistant｜>```lean4
theorem Equation2_implies_Equation4 (G: Type _) [Magma G] (h: Equation2 G) : Equation4 G := by
  intro x y
  have h1 := h (x ◇ y) (x ◇ y)
  have h2 := h x y
  have h3 := h y x
  have h4 := h (x ◇ y) x
  have h5 := h (x ◇ y) y
  have h6 := h x (x ◇ y)
  have h7 := h y (x ◇ y)
  -- Simplify the expressions using the given equality x = y
  simp_all
```<｜end▁of▁sentence｜>
2.5087499618530273


In [12]:
prompt = """
Complete the following Lean 4 code:

```lean4
{}
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.
""".strip()

chat = [
  {"role": "user", "content": prompt.format(formal_statement)},
]
inputs = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)

In [13]:
import time
start = time.time()
outputs = model.generate(inputs, max_new_tokens=8192)
print(tokenizer.batch_decode(outputs)[0])
print(time.time() - start)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Complete the following Lean 4 code:

```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

class Magma (α : Type _) where
  op : α → α → α

infix:65 " ◇ " => Magma.op

abbrev Equation2 (G: Type _) [Magma G] := ∀ x y : G, x = y

abbrev Equation4 (G: Type _) [Magma G] := ∀ x y : G, x = x ◇ y

theorem Equation2_implies_Equation4 (G: Type _) [Magma G] (h: Equation2 G) : Equation4 G := by
  sorry
```

Before producing the Lean 4 code to formally prove the given theorem, provide a detailed proof plan outlining the main proof steps and strategies.
The plan should highlight key ideas, intermediate lemmas, and proof structures that will guide the construction of the final formal proof.<｜Assistant｜>### Detailed Proof and Analysis

First, let's understand the problem:
1. We have a type `G` and a `Magma` structure on `G`, i.e., a binary operation `◇` on `G`.
2. The hypothesis `h : Equation2 G` means that for all `x y

In [9]:
output = """
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

class Magma (α : Type _) where
  op : α → α → α

infix:65 " ◇ " => Magma.op

abbrev Equation2 (G: Type _) [Magma G] := ∀ x y : G, x = y

abbrev Equation4 (G: Type _) [Magma G] := ∀ x y : G, x = x ◇ y

theorem Equation2_implies_Equation4 (G: Type _) [Magma G] (h: Equation2 G) : Equation4 G := by
  sorry
```<｜Assistant｜>```lean4
theorem Equation2_implies_Equation4 (G: Type _) [Magma G] (h: Equation2 G) : Equation4 G := by
  intro x y
  have h1 := h (x ◇ y) (x ◇ y)
...
  -- Simplify the expressions using the given equality x = y
  simp_all
```<｜end▁of▁sentence｜>
"""
response = output.split("```lean4")[-1].split("```")[0].strip()


In [12]:
statement = """
import Mathlib.Tactic

class Magma (α : Type _) where
  op : α → α → α

infix:65 " ◇ " => Magma.op

abbrev Equation837 (G: Type _) [Magma G] := ∀ x y z : G, x = x ◇ ((y ◇ x) ◇ (y ◇ z))

abbrev Equation3 (G: Type _) [Magma G] := ∀ x : G, x = x ◇ x

theorem Equation837_not_implies_Equation3 : ∃ (G: Type) (_: Magma G), Equation837 G ∧ ¬ Equation3 G := by
  sorry
"""
statement_prefix = statement.split("theorem")[0].strip()
print(statement_prefix + '\n\n' + response)

import Mathlib.Tactic

class Magma (α : Type _) where
  op : α → α → α

infix:65 " ◇ " => Magma.op

abbrev Equation837 (G: Type _) [Magma G] := ∀ x y z : G, x = x ◇ ((y ◇ x) ◇ (y ◇ z))

abbrev Equation3 (G: Type _) [Magma G] := ∀ x : G, x = x ◇ x

theorem Equation2_implies_Equation4 (G: Type _) [Magma G] (h: Equation2 G) : Equation4 G := by
  intro x y
  have h1 := h (x ◇ y) (x ◇ y)
...
  -- Simplify the expressions using the given equality x = y
  simp_all
